In [1]:

import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import random
import os
from google.colab import drive


# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

def partition_drug_interactions(input_path, output_base_path, seeds=[32, 42, 46],
                                                   train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    """
    Partition drug interaction dataset with GUARANTEED no cold start problem

    Strategy:
    1. Ensure ALL drugs appear in training first
    2. Then distribute remaining interactions to val/test
    """

    print("="*80)
    print("DRUG INTERACTION DATASET PARTITIONING - FIXED NO COLD START")
    print("="*80)

    # Load the complete dataset
    print(f" Loading data from: {input_path}")
    df = pd.read_csv(input_path)

    print(f" Total interactions: {len(df):,}")

    # Get all unique drugs
    all_drugs = set(df['Drug1_ID'].unique()) | set(df['Drug2_ID'].unique())
    all_drugs = list(all_drugs)
    print(f" Total unique drugs: {len(all_drugs):,}")

    # Calculate target split sizes
    total_samples = len(df)
    target_train_size = int(total_samples * train_ratio)
    target_val_size = int(total_samples * val_ratio)
    target_test_size = total_samples - target_train_size - target_val_size

    print(f"\n TARGET SPLIT SIZES:")
    print(f"Training: {target_train_size:,} samples ({train_ratio*100:.1f}%)")
    print(f"Validation: {target_val_size:,} samples ({val_ratio*100:.1f}%)")
    print(f"Test: {target_test_size:,} samples ({test_ratio*100:.1f}%)")

    # Process each seed
    for seed in seeds:
        print(f"\n" + "="*60)
        print(f" PROCESSING SEED {seed}")
        print("="*60)

        # Create output directory
        seed_output_dir = os.path.join(output_base_path, f'train-seed{seed}')
        os.makedirs(seed_output_dir, exist_ok=True)

        # Step 1: Shuffle the dataset
        df_shuffled = shuffle(df, random_state=seed).reset_index(drop=True) # =true ; Discards the old index instead of adding it as a new column

        # Step 2: Ensure all drugs appear in training
        print("🔍 Step 1: Ensuring all drugs appear in training...")

        # Track which drugs we've seen in training
        drugs_in_training = set()
        guaranteed_training_indices = [] #If both drugs/one of them fisrt seen /exist in training
        remaining_indices = [] #If both drugs already exist in training

        # Go through shuffled data and collect interactions until all drugs are covered
        for idx, row in df_shuffled.iterrows():
            drug1, drug2 = row['Drug1_ID'], row['Drug2_ID']

            # If we haven't seen both drugs in training yet, add to guaranteed training
            if drug1 not in drugs_in_training or drug2 not in drugs_in_training:
                guaranteed_training_indices.append(idx)
                drugs_in_training.add(drug1)
                drugs_in_training.add(drug2)
            else:
                remaining_indices.append(idx)

            # Check if we have all drugs || Early stop optimization; Once all drugs are covered, stop looping
            if len(drugs_in_training) == len(all_drugs):
                # Add remaining indices to remaining_indices
                remaining_indices.extend(range(idx+1, len(df_shuffled)))
                break

        print(f"    Guaranteed training interactions (covers all drugs): {len(guaranteed_training_indices):,}")
        print(f"    Remaining interactions for distribution: {len(remaining_indices):,}")
        print(f"    Drugs covered in training: {len(drugs_in_training):,}/{len(all_drugs):,}")

        # Step 3: Create DataFrames from indices
        guaranteed_train_df = df_shuffled.iloc[guaranteed_training_indices].copy()
        remaining_df = df_shuffled.iloc[remaining_indices].copy() if remaining_indices else pd.DataFrame(columns=df.columns)

        # Shuffle remaining interactions
        if len(remaining_df) > 0:
            remaining_df = shuffle(remaining_df, random_state=seed + 1000).reset_index(drop=True)

        # Calculate how many more samples we need for each split
        guaranteed_train_size = len(guaranteed_training_indices)
        additional_train_needed = max(0, target_train_size - guaranteed_train_size)

        print(f"\n DISTRIBUTION CALCULATION:")
        print(f"   Guaranteed training samples: {guaranteed_train_size:,}")
        print(f"   Additional training needed: {additional_train_needed:,}")
        print(f"   Remaining for val/test distribution: {len(remaining_indices):,}")

        # Distribute remaining interactions
        if len(remaining_df) > 0:
            # Take additional training samples if needed
            additional_train_end = min(additional_train_needed, len(remaining_df))
            additional_train_df = remaining_df.iloc[:additional_train_end].copy()

            # Distribute the rest to val and test
            remaining_for_val_test = remaining_df.iloc[additional_train_end:].copy()
            val_size_from_remaining = min(target_val_size, len(remaining_for_val_test))

            val_df_temp = remaining_for_val_test.iloc[:val_size_from_remaining].copy()
            test_df_temp = remaining_for_val_test.iloc[val_size_from_remaining:].copy()
        else:
             #creates empty DataFrames
            additional_train_df = pd.DataFrame(columns=df.columns)
            val_df_temp = pd.DataFrame(columns=df.columns)
            test_df_temp = pd.DataFrame(columns=df.columns)

        # Step 4: Combine final splits
        train_df_final = pd.concat([guaranteed_train_df, additional_train_df], ignore_index=True)
        val_df_final = val_df_temp.copy()
        test_df_final = test_df_temp.copy()

        # Shuffle final datasets
        train_df_final = shuffle(train_df_final, random_state=seed + 2000).reset_index(drop=True) # or can use just seed without any plus value like 2000 or 3000
        if len(val_df_final) > 0: val_df_final = shuffle(val_df_final, random_state=seed + 3000).reset_index(drop=True)
        else:
          break  # WRONG!
        if len(test_df_final) > 0:
            test_df_final = shuffle(test_df_final, random_state=seed + 4000).reset_index(drop=True)
        else:
          break  # WRONG!

        # Step 5: Final verification
        train_drugs_final = set(train_df_final['Drug1_ID']) | set(train_df_final['Drug2_ID'])
        val_drugs_final = set(val_df_final['Drug1_ID']) | set(val_df_final['Drug2_ID']) if len(val_df_final) > 0 else set()
        test_drugs_final = set(test_df_final['Drug1_ID']) | set(test_df_final['Drug2_ID']) if len(test_df_final) > 0 else set()


        print(f"\n FINAL VERIFICATION:")
        print(f"   Training samples: {len(train_df_final):,}")
        print(f"   Validation samples: {len(val_df_final):,}")
        print(f"   Test samples: {len(test_df_final):,}")
        print(f"   Total samples: {len(train_df_final) + len(val_df_final) + len(test_df_final):,}")
        print(f"   Training drugs: {len(train_drugs_final):,}")
        print(f"   Validation drugs: {len(val_drugs_final):,}")
        print(f"   Test drugs: {len(test_drugs_final):,}")

        # Step 6: Save the files
        train_path = os.path.join(seed_output_dir, 'train.csv')
        val_path = os.path.join(seed_output_dir, 'val.csv')
        test_path = os.path.join(seed_output_dir, 'test.csv')

        train_df_final.to_csv(train_path, index=False)
        val_df_final.to_csv(val_path, index=False)
        test_df_final.to_csv(test_path, index=False)

        print(f"\n FILES SAVED:")
        print(f"   {train_path}")
        print(f"   {val_path}")
        print(f"   {test_path}")

        # Verify all drugs are in training
        if len(train_drugs_final) == len(all_drugs):
            print(f"    SUCCESS: All {len(all_drugs):,} drugs are present in training!")
        else:
            print(f"    ERROR: Only {len(train_drugs_final):,}/{len(all_drugs):,} drugs in training!")
            missing_drugs = set(all_drugs) - train_drugs_final
            print(f"   Missing drugs: {list(missing_drugs)[:10]}...")

# Define paths
input_path = '/content/drive/MyDrive/MLHygnn/DB/DataSetOf1709Drugs/DDI_unique_interactionsAnalysis.csv'
output_base_path = '/content/drive/MyDrive/MLHygnn/DB/Partition-Dataset4'

# Define seeds and ratios
seeds = [32, 42, 46]
train_ratio = 0.8  # 80%
val_ratio = 0.1    # 10%
test_ratio = 0.1   # 10%

print(" Starting FIXED drug interaction dataset partitioning...")
print(f" Target ratios: {train_ratio*100:.0f}% train, {val_ratio*100:.0f}% validation, {test_ratio*100:.0f}% test")
print(f" Seeds: {seeds}")
print(" Strategy: Guarantee all drugs in training first, then distribute remaining interactions")

# Run partitioning
partition_drug_interactions(
    input_path=input_path,
    output_base_path=output_base_path,
    seeds=seeds,
    train_ratio=train_ratio,
    val_ratio=val_ratio,
    test_ratio=test_ratio
)

print(f"\n FIXED PARTITIONING COMPLETED!")
print(" GUARANTEED: No cold start problem - all drugs in validation/test also appear in training!")

 Starting FIXED drug interaction dataset partitioning...
 Target ratios: 80% train, 10% validation, 10% test
 Seeds: [32, 42, 46]
 Strategy: Guarantee all drugs in training first, then distribute remaining interactions
DRUG INTERACTION DATASET PARTITIONING - FIXED NO COLD START
 Loading data from: /content/drive/MyDrive/MLHygnn/DB/DataSetOf1709Drugs/DDI_unique_interactionsAnalysis.csv
 Total interactions: 191,877
 Total unique drugs: 1,709

 TARGET SPLIT SIZES:
Training: 153,501 samples (80.0%)
Validation: 19,187 samples (10.0%)
Test: 19,189 samples (10.0%)

 PROCESSING SEED 32
🔍 Step 1: Ensuring all drugs appear in training...
    Guaranteed training interactions (covers all drugs): 1,435
    Remaining interactions for distribution: 190,442
    Drugs covered in training: 1,709/1,709

 DISTRIBUTION CALCULATION:
   Guaranteed training samples: 1,435
   Additional training needed: 152,066
   Remaining for val/test distribution: 190,442

 FINAL VERIFICATION:
   Training samples: 153,501
 


### Note on Drugs Appearing Only in the Training Split

We observed that some drugs appear exclusively in the training partition and do not show up in the validation or test sets. This happens when a drug has **only one interaction** across the entire dataset. In these cases, we assign that interaction to the training split so the model can still learn from the drug’s chemical structure.
This ensures the neural network gains exposure to all available drug features during training, even when interaction data is extremely sparse.


In [9]:
def analyze_drugs_across_splits(train_path, val_path, test_path):

    print("="*70)
    print("DRUG ANALYSIS ACROSS ALL SPLITS")
    print("="*70)

    # Load all splits
    print("Loading data...")
    train_df = pd.read_csv(train_path)
    val_df = pd.read_csv(val_path)
    test_df = pd.read_csv(test_path)

    print(f" Training samples: {len(train_df)}")
    print(f" Validation samples: {len(val_df)}")
    print(f" Test samples: {len(test_df)}")
    print(f" Total positive samples: {len(train_df) + len(val_df) + len(test_df)}")

    # Get unique drugs from each split
    train_drugs = set(train_df['Drug1_ID'].unique()) | set(train_df['Drug2_ID'].unique())
    val_drugs = set(val_df['Drug1_ID'].unique()) | set(val_df['Drug2_ID'].unique())
    test_drugs = set(test_df['Drug1_ID'].unique()) | set(test_df['Drug2_ID'].unique())

    print(f"\n DRUGS PER SPLIT:")
    print(f"Training split drugs: {len(train_drugs)}")
    print(f"Validation split drugs: {len(val_drugs)}")
    print(f"Test split drugs: {len(test_drugs)}")

    # Find total unique drugs across ALL splits
    all_drugs = train_drugs | val_drugs | test_drugs
    print(f"\n TOTAL UNIQUE DRUGS ACROSS ALL SPLITS: {len(all_drugs)}")

    # Analyze drug overlap between splits
    print(f"\n DRUG OVERLAP ANALYSIS:")

    # Drugs only in training
    train_only = train_drugs - val_drugs - test_drugs
    print(f"Drugs only in Training: {len(train_only)}")

    # Drugs only in validation
    val_only = val_drugs - train_drugs - test_drugs
    print(f"Drugs only in Validation: {len(val_only)}")

    # Drugs only in test
    test_only = test_drugs - train_drugs - val_drugs
    print(f"Drugs only in Test: {len(test_only)}")

    # Drugs in all three splits
    all_three = train_drugs & val_drugs & test_drugs
    print(f"Drugs in ALL three splits: {len(all_three)}")

    # Drugs in training + validation only
    train_val_only = (train_drugs & val_drugs) - test_drugs
    print(f"Drugs in Training + Validation only: {len(train_val_only)}")

    # Drugs in training + test only
    train_test_only = (train_drugs & test_drugs) - val_drugs
    print(f"Drugs in Training + Test only: {len(train_test_only)}")

    # Drugs in validation + test only
    val_test_only = (val_drugs & test_drugs) - train_drugs
    print(f"Drugs in Validation + Test only: {len(val_test_only)}")

    # Verification
    total_check = (len(train_only) + len(val_only) + len(test_only) +
                   len(all_three) + len(train_val_only) + len(train_test_only) + len(val_test_only))

    print(f"\n VERIFICATION:")
    print(f"Sum of all categories: {total_check}")
    print(f"Total unique drugs: {len(all_drugs)}")
    print(f"Match: {' YES' if total_check == len(all_drugs) else ' NO'}")

    # Show some examples if there are drugs missing from training
    if train_only or val_only or test_only:
        print(f"\n  WARNING: Some drugs don't appear in all splits!")
        print("This might affect negative sampling if we only use drugs from individual splits.")

        if val_only:
            print(f"\nExample drugs only in Validation: {list(val_only)[:5]}")
        if test_only:
            print(f"Example drugs only in Test: {list(test_only)[:5]}")
        if train_only:
            print(f"Example drugs only in Training: {list(train_only)[:5]}")

    print(f"\n" + "="*70)

    return {
        'all_drugs': all_drugs,
        'train_drugs': train_drugs,
        'val_drugs': val_drugs,
        'test_drugs': test_drugs,
        'total_unique_drugs': len(all_drugs)
    }

train_path = '/content/drive/MyDrive/MLHygnn/DB/Partition-Dataset/train-seed42/train_accurate_redistribution.csv'
val_path = '/content/drive/MyDrive/MLHygnn/DB/Partition-Dataset/train-seed42/val_accurate_redistribution.csv'
test_path = '/content/drive/MyDrive/MLHygnn/DB/Partition-Dataset/train-seed42/test_accurate_redistribution.csv'

# Run analysis
drug_analysis = analyze_drugs_across_splits(train_path, val_path, test_path)

print(f"\n FINAL ANSWER: Your dataset contains {drug_analysis['total_unique_drugs']} unique drugs total.")
print("If this doesn't match your expected 1,694, we need to investigate further!")

DRUG ANALYSIS ACROSS ALL SPLITS
Loading data...
 Training samples: 153501
 Validation samples: 19187
 Test samples: 19189
 Total positive samples: 191877

 DRUGS PER SPLIT:
Training split drugs: 1709
Validation split drugs: 1536
Test split drugs: 1540

 TOTAL UNIQUE DRUGS ACROSS ALL SPLITS: 1709

 DRUG OVERLAP ANALYSIS:
Drugs only in Training: 121
Drugs only in Validation: 0
Drugs only in Test: 0
Drugs in ALL three splits: 1488
Drugs in Training + Validation only: 48
Drugs in Training + Test only: 52
Drugs in Validation + Test only: 0

 VERIFICATION:
Sum of all categories: 1709
Total unique drugs: 1709
Match:  YES

This might affect negative sampling if we only use drugs from individual splits.
Example drugs only in Training: ['DB00832', 'DB09014', 'DB00198', 'DB01073', 'DB09135']


 FINAL ANSWER: Your dataset contains 1709 unique drugs total.
If this doesn't match your expected 1,694, we need to investigate further!


# **Partition metabolic data**





### Metabolic Network Data Partitioning Overview

This section describes the workflow used to partition interaction data for the metabolic neural network. The partitioning is performed **by interaction type** to maintain consistent and balanced splits.

1. **Load Previous Partitions**
   We begin by loading the existing training, validation, and testing partitions.

2. **Evaluate Partition Balance**
   We run the `analyze_partition_balance` function to inspect how each interaction type is distributed across the three splits. This step highlights any interaction types that are poorly balanced or incorrectly assigned.

3. **Extract Problematic Types**
   Using the `data-partition-metabolic` function, we isolate only the problematic interaction types from the current partitions. All correctly distributed interactions remain unchanged.

4. **Repartition Problematic Interactions**
   The extracted types are repartitioned to achieve the correct distribution across train/validation/test.

5. **Re-verify Partitioning**
   We rerun the analysis to confirm that the updated partitioning is correct.
   This workflow is repeated for all remaining random seeds(42,32).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import random
import os
import pandas as pd
from sklearn.model_selection import train_test_split

## Data partation in correct

**fun data-partation-metabolic 👇**

In [ ]:

base_path = '/content/drive/MyDrive/MLHygnn/DB/Partition-Dataset/Metabolic-Seed42/'
train_path = base_path + 'train_metabolic.csv'
val_path = base_path + 'val_metabolic.csv'
test_path = base_path + 'test_metabolic.csv'

# PROBLEMATIC LABELS
problematic_labels = [1,26,41,42,44,52,86,79]  # All problematic labels that get from (analyze_partition_balance)

print("="*60)
print("SIMPLE LABEL REDISTRIBUTION")
print("="*60)
print(f"\nProblematic labels to fix: {problematic_labels}")

# Read files
print("\nReading files...")
train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)
test_df = pd.read_csv(test_path)
print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# Extract problematic rows from all files
print("\nExtracting problematic rows...")
train_prob = train_df[train_df['Label'].isin(problematic_labels)]
val_prob = val_df[val_df['Label'].isin(problematic_labels)]
test_prob = test_df[test_df['Label'].isin(problematic_labels)]

print(f"From Train: {len(train_prob)} rows")
print(f"From Val: {len(val_prob)} rows")
print(f"From Test: {len(test_prob)} rows")

# Keep non-problematic rows unchanged
train_keep = train_df[~train_df['Label'].isin(problematic_labels)]
val_keep = val_df[~val_df['Label'].isin(problematic_labels)]
test_keep = test_df[~test_df['Label'].isin(problematic_labels)]

# Combine all problematic rows
combined_prob = pd.concat([train_prob, val_prob, test_prob], ignore_index=True)
print(f"\nTotal problematic rows to redistribute: {len(combined_prob)}")

# Split: 80% train, 10% val, 10% test
print("\nRedistributing: 80% train, 10% val, 10% test...")

# Check if we can use stratification
label_counts = combined_prob['Label'].value_counts()
min_count = label_counts.min()

print(f"Minimum samples per label: {min_count}")

if min_count >= 10:
    # Enough samples for stratified split
    print("Using stratified split...")
    train_new, temp = train_test_split(
        combined_prob,
        test_size=0.2, # Keep 20% for val+test
        stratify=combined_prob['Label'],
        random_state=32
    )

    val_new, test_new = train_test_split(
        temp,
        test_size=0.5,  # Split temp 50/50
        stratify=temp['Label'],
        random_state=32
    )
else:
    # Manual split for small labels
    print("Using manual split (labels have too few samples for stratification)...")
    train_new = []
    val_new = []
    test_new = []

    for label in problematic_labels:
        label_data = combined_prob[combined_prob['Label'] == label].copy()
        n = len(label_data)

        # Shuffle
        label_data = label_data.sample(frac=1, random_state=32).reset_index(drop=True)

        # Calculate splits: 80% train, 10% val, 10% test
        n_train = max(1, int(n * 0.8))
        n_val = max(1, int(n * 0.1))
        n_test = n - n_train - n_val

        # Ensure at least 1 in test if there are samples left
        if n_test < 1 and n > n_train + n_val:
            n_test = 1
            n_val = n - n_train - n_test

        train_new.append(label_data.iloc[:n_train])
        val_new.append(label_data.iloc[n_train:n_train+n_val])
        test_new.append(label_data.iloc[n_train+n_val:])

    train_new = pd.concat(train_new, ignore_index=True)
    val_new = pd.concat(val_new, ignore_index=True)
    test_new = pd.concat(test_new, ignore_index=True)

print(f"New distribution: Train={len(train_new)}, Val={len(val_new)}, Test={len(test_new)}")

# Combine with kept rows
train_final = pd.concat([train_keep, train_new], ignore_index=True)
val_final = pd.concat([val_keep, val_new], ignore_index=True)
test_final = pd.concat([test_keep, test_new], ignore_index=True)

print(f"\nFinal sizes: Train={len(train_final)}, Val={len(val_final)}, Test={len(test_final)}")

# Show distribution of fixed labels
print("\nNew distribution of problematic labels:")
print(f"{'Label':<10} {'Train':<10} {'Val':<10} {'Test':<10}")
print("-"*40)
for label in problematic_labels:
    t = len(train_final[train_final['Label'] == label])
    v = len(val_final[val_final['Label'] == label])
    te = len(test_final[test_final['Label'] == label])
    print(f"{label:<10} {t:<10} {v:<10} {te:<10}")

# Save files
import os
output_path = base_path + 'fixed/'
os.makedirs(output_path, exist_ok=True)

train_final.to_csv(output_path + 'train_fixed.csv', index=False)
val_final.to_csv(output_path + 'val_fixed.csv', index=False)
test_final.to_csv(output_path + 'test_fixed.csv', index=False)

print(f"\n✓ Files saved to: {output_path}")
print("  - train_fixed.csv")
print("  - val_fixed.csv")
print("  - test_fixed.csv")
print("\n✓ Done!")

SIMPLE LABEL REDISTRIBUTION

Problematic labels to fix: [1, 26, 41, 42, 44, 52, 86, 79]

Reading files...
Train: 153501, Val: 19187, Test: 19189

Extracting problematic rows...
From Train: 100 rows
From Val: 9 rows
From Test: 6 rows

Total problematic rows to redistribute: 115

Redistributing: 80% train, 10% val, 10% test...
Minimum samples per label: 6
Using manual split (labels have too few samples for stratification)...
New distribution: Train=88, Val=10, Test=17

Final sizes: Train=153489, Val=19188, Test=19200

New distribution of problematic labels:
Label      Train      Val        Test      
----------------------------------------
1          8          1          2         
26         5          1          1         
41         11         1          2         
42         4          1          1         
44         10         1          2         
52         8          1          1         
86         21         2          4         
79         21         2          4         



In [ ]:
def analyze_partition_balance(partition_base_path, seeds=[32, 42]):

    print("PARTITION BALANCE VERIFICATION")
    print("="*70)

    all_results = {}

    for seed in seeds:
        print(f"\nSEED {seed} ANALYSIS")
        print("-" * 50)

        # Define file paths
        seed_dir = '/content/drive/MyDrive/MLHygnn/DB/Partition-Dataset/Metabolic-Seed42/fixed/'
        train_path = os.path.join(seed_dir, 'train_fixed.csv')
        val_path = os.path.join(seed_dir, 'val_fixed.csv')
        test_path = os.path.join(seed_dir, 'test_fixed.csv')

        # Load data
        train_df = pd.read_csv(train_path)
        val_df = pd.read_csv(val_path)
        test_df = pd.read_csv(test_path)

        # Convert Label to integer to avoid format issues
        train_df['Label'] = train_df['Label'].astype(int)
        val_df['Label'] = val_df['Label'].astype(int)
        test_df['Label'] = test_df['Label'].astype(int)

        print(f"Train samples: {len(train_df):,}")
        print(f"Val samples: {len(val_df):,}")
        print(f"Test samples: {len(test_df):,}")

        # Count interaction types in each split
        train_counts = train_df['Label'].value_counts().sort_index()
        val_counts = val_df['Label'].value_counts().sort_index()
        test_counts = test_df['Label'].value_counts().sort_index()

        # Create comprehensive analysis
        all_labels = sorted(set(train_df['Label'].unique()) |
                           set(val_df['Label'].unique()) |
                           set(test_df['Label'].unique()))

        analysis_data = []

        for label in all_labels:
            train_count = int(train_counts.get(label, 0))
            val_count = int(val_counts.get(label, 0))
            test_count = int(test_counts.get(label, 0))
            total_count = train_count + val_count + test_count

            # Calculate percentages
            train_pct = (train_count / total_count * 100) if total_count > 0 else 0
            val_pct = (val_count / total_count * 100) if total_count > 0 else 0
            test_pct = (test_count / total_count * 100) if total_count > 0 else 0

            analysis_data.append({
                'Interaction_Type': int(label),
                'Train_Count': train_count,
                'Val_Count': val_count,
                'Test_Count': test_count,
                'Total_Count': total_count,
                'Train_Pct': round(train_pct, 1),
                'Val_Pct': round(val_pct, 1),
                'Test_Pct': round(test_pct, 1)
            })

        # Convert to DataFrame for analysis
        analysis_df = pd.DataFrame(analysis_data)
        all_results[seed] = analysis_df

        # Summary statistics
        print(f"\nDISTRIBUTION SUMMARY:")
        print(f"Total interaction types: {len(all_labels)}")
        print(f"Types missing from train: {len([l for l in all_labels if train_counts.get(l, 0) == 0])}")
        print(f"Types missing from val: {len([l for l in all_labels if val_counts.get(l, 0) == 0])}")
        print(f"Types missing from test: {len([l for l in all_labels if test_counts.get(l, 0) == 0])}")

        # Identify problematic distributions
        problematic = analysis_df[
            (analysis_df['Train_Pct'] < 70) |
            (analysis_df['Train_Pct'] > 90) |
            (analysis_df['Val_Count'] == 0) |
            (analysis_df['Test_Count'] == 0)
        ]

        if len(problematic) > 0:
            print(f"\nPROBLEMATIC DISTRIBUTIONS ({len(problematic)} types):")
            print("Type | Train% | Val% | Test% | Train_Count | Val_Count | Test_Count")
            print("-" * 70)
            for _, row in problematic.iterrows():
                print(f"{int(row['Interaction_Type'])} | {row['Train_Pct']}% | {row['Val_Pct']}% | {row['Test_Pct']}% | {int(row['Train_Count'])} | {int(row['Val_Count'])} | {int(row['Test_Count'])}")
        else:
            print("\nNo problematic distributions detected!")

        # Top 10 most common types
        top_10 = analysis_df.nlargest(10, 'Total_Count')
        print(f"\nTOP 10 MOST COMMON INTERACTION TYPES:")
        print("Type | Total_Count | Train | Val | Test | Train%")
        print("-" * 55)
        for _, row in top_10.iterrows():
            print(f"{int(row['Interaction_Type'])} | {int(row['Total_Count'])} | {int(row['Train_Count'])} | {int(row['Val_Count'])} | {int(row['Test_Count'])} | {row['Train_Pct']}%")

        # Bottom 10 least common types
        bottom_10 = analysis_df.nsmallest(10, 'Total_Count')
        print(f"\nBOTTOM 10 LEAST COMMON INTERACTION TYPES:")
        print("Type | Total_Count | Train | Val | Test | Train%")
        print("-" * 55)
        for _, row in bottom_10.iterrows():
            print(f"{int(row['Interaction_Type'])} | {int(row['Total_Count'])} | {int(row['Train_Count'])} | {int(row['Val_Count'])} | {int(row['Test_Count'])} | {row['Train_Pct']}%")

        # Save detailed report
        output_path = os.path.join(seed_dir, 'balance_analysis.csv')
        analysis_df.to_csv(output_path, index=False)
        print(f"\nDetailed analysis saved to: {output_path}")

    print(f"\n" + "="*70)
    print("CROSS-SEED COMPARISON")
    print("="*70)

    # Compare consistency across seeds
    if len(all_results) > 1:
        # Check if the same types are problematic across seeds
        problematic_types = set()
        for seed, df in all_results.items():
            problematic = df[
                (df['Train_Pct'] < 70) |
                (df['Train_Pct'] > 90) |
                (df['Val_Count'] == 0) |
                (df['Test_Count'] == 0)
            ]
            problematic_types.update(problematic['Interaction_Type'].tolist())

        if problematic_types:
            print(f"Interaction types with issues across any seed: {sorted(list(problematic_types))}")
            print("These types may need special attention during training (class weighting, etc.)")
        else:
            print("No consistent distribution problems across seeds!")

        # Calculate coefficient of variation for train percentages across seeds
        train_pct_variations = {}
        for interaction_type in all_labels:
            train_pcts = []
            for seed, df in all_results.items():
                row = df[df['Interaction_Type'] == interaction_type]
                if len(row) > 0:
                    train_pcts.append(row['Train_Pct'].iloc[0])

            if len(train_pcts) > 1:
                cv = np.std(train_pcts) / np.mean(train_pcts) if np.mean(train_pcts) > 0 else 0
                train_pct_variations[interaction_type] = cv

        # Show types with high variation across seeds
        high_variation = {k: v for k, v in train_pct_variations.items() if v > 0.1}
        if high_variation:
            print(f"\nTypes with high train% variation across seeds:")
            for interaction_type, cv in sorted(high_variation.items(), key=lambda x: x[1], reverse=True):
                print(f"  Type {interaction_type}: CV = {cv:.3f}")
        else:
            print("\nConsistent train% distributions across all seeds!")

    print("\nANALYSIS COMPLETE!")
    return all_results


# Usage
partition_base_path = '/content/drive/MyDrive/MLHygnn/DB/Partition-Dataset/'
seeds = [32]

results = analyze_partition_balance(partition_base_path, seeds)

PARTITION BALANCE VERIFICATION

SEED 32 ANALYSIS
--------------------------------------------------
Train samples: 153,489
Val samples: 19,188
Test samples: 19,200

DISTRIBUTION SUMMARY:
Total interaction types: 86
Types missing from train: 0
Types missing from val: 0
Types missing from test: 0

PROBLEMATIC DISTRIBUTIONS (1 types):
Type | Train% | Val% | Test% | Train_Count | Val_Count | Test_Count
----------------------------------------------------------------------
42 | 66.7% | 16.7% | 16.7% | 4 | 1 | 1

TOP 10 MOST COMMON INTERACTION TYPES:
Type | Total_Count | Train | Val | Test | Train%
-------------------------------------------------------
49 | 60971 | 48746 | 6126 | 6099 | 79.9%
47 | 34359 | 27490 | 3464 | 3405 | 80.0%
73 | 23540 | 18789 | 2345 | 2406 | 79.8%
75 | 9393 | 7439 | 993 | 961 | 79.2%
60 | 8393 | 6728 | 834 | 831 | 80.2%
70 | 7787 | 6217 | 762 | 808 | 79.8%
20 | 6140 | 4927 | 617 | 596 | 80.2%
16 | 5435 | 4384 | 537 | 514 | 80.7%
4 | 5011 | 3996 | 511 | 504 | 79.7%
